# Visualize Simulation Data
Visualize different components/processed versions of the simulation data using the extremely customizable pygwalker library

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from inverse_modelling_tfo.data import (
    generate_data_loaders,
    equidistance_detector_normalization,
    constant_detector_count_normalization,
)
from inverse_modelling_tfo.data.intensity_interpolation import (
    interpolate_exp,
    get_interpolate_fit_params,
    exp_piecewise_affine,
)
from inverse_modelling_tfo.data.interpolation_function_zoo import *
import pygwalker as pyg
from inverse_modelling_tfo.features.build_features import FetalACFeatureBuilder
from inverse_modelling_tfo.features.data_transformations import LongToWideIntensityTransformation

In [18]:
# DATA_PATH = r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/s_based_intensity_low_conc3.pkl'
# DATA_PATH = (
#     r"/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/tools/s_based_intensity_low_conc5.pkl"
# )

# Using pencil source + equispaced detector(corrected) + 1e9 photons
DATA_PATH = r'/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/dan_iccps_pencil.pkl'

data = pd.read_pickle(DATA_PATH)
# Drop Thickness values which I did not calculate the close concnetration points
equidistance_detector_normalization(data)

# Drop Uterus Thickness for now
data = data.drop(columns="Uterus Thickness")

# Interpolate intensity to remove noise
# data = interpolate_exp(data, weights=(1, 0.8), interpolation_function=exp_piecewise_affine, break_indices=[4, 12, 20])
# data['Intensity'] = data['Interpolated Intensity']
# data = data.drop(columns='Interpolated Intensity')

data_transformer = LongToWideIntensityTransformation()
data = data_transformer.transform(data)
labels = data_transformer.get_label_names()
intensity_columns = data_transformer.get_feature_names()

# Manual log(intensity) normalization
# data['Intensity'] = np.log10(data['Intensity'])        # Far values wayy to small to affect anything. Take log


# data, x_columns, labels = create_ratio_and_intensity(data, True)
# # data, x_columns, labels = create_ratio(data, True)
# data, x_columns, labels = create_spatial_intensity(data)

In [19]:
# Round out the data points
data["Maternal Hb Concentration"] = data["Maternal Hb Concentration"].round(2)
data["Fetal Hb Concentration"] = data["Fetal Hb Concentration"].round(2)
data["Maternal Saturation"] = data["Maternal Saturation"].round(2)
data["Fetal Saturation"] = data["Fetal Saturation"].round(2)

In [20]:
# Filter data so it does not crash the visualization
data = data[(data["Maternal Hb Concentration"] == 11.0) & (data["Maternal Saturation"] == 1.0)]   

In [21]:
# fetal_conc_group_mapping = {
#     10.45 : 0,
#     11.00: 0, 
#     11.40: 0,
#     11.55: 1,
#     12.00: 1,
#     12.35: 1,
#     12.60: 2,
#     13.00: 2,
#     13.30: 2,
#     13.65: 3,
#     14.00: 3,
#     14.25: 3,
#     14.70: 4,
#     15.00: 4,
#     15.75: 4,
# }
# data['FconcCenters'] = data['Fetal Hb Concentration'].round(2).map(fetal_conc_group_mapping)

In [22]:
fetal_conc_group_mapping = {
    10.45 : 0,
    10.88 : 0,
    11. : 0,
    11.31: 1,
    11.45: 1,
    11.55: 1,
    11.75: 2,
    11.91: 2,
    12.03: 2,
    12.18: 3,
    12.36: 3,
    12.5: 3,
    12.61: 4,
    12.82: 4,
    12.98: 4,
    13.04: 5,
    13.27: 5,
    13.46: 5,
    13.47: 6,
    13.73: 6,
    13.9: 6,
    13.94: 7,
    14.18: 7,
    14.34: 7,
    14.41: 8,
    14.64: 8,
    14.77: 8,
    14.89: 9,
    15.09: 9,
    15.2: 9,
    15.37: 10,
    15.55: 10,
    15.85: 10,
    16.: 11,
    16.32: 11,
    16.8: 11, 
}
data['FconcCenters'] = data['Fetal Hb Concentration'].round(2).map(fetal_conc_group_mapping)

In [23]:
# fetal_ac_feature_builder = FetalACFeatureBuilder('FconcCenters',  'comb', '/')
fetal_ac_feature_builder = FetalACFeatureBuilder("FconcCenters", "comb", "/", intensity_columns, labels)
# fetal_ac_feature_builder = FetalACFeatureBuilder('FconcCenters', True, 'comb')
data = fetal_ac_feature_builder.build_feature(data)

In [24]:
data.columns

Index(['Maternal Wall Thickness', 'Maternal Hb Concentration',
       'Maternal Saturation', 'Fetal Saturation', 'FconcCenters',
       'Fetal Hb Concentration 0', 'Fetal Hb Concentration 1', 'DivAC_WV1_0',
       'DivAC_WV2_0', 'DivAC_WV1_1', 'DivAC_WV2_1', 'DivAC_WV1_2',
       'DivAC_WV2_2', 'DivAC_WV1_3', 'DivAC_WV2_3', 'DivAC_WV1_4',
       'DivAC_WV2_4', 'DivAC_WV1_5', 'DivAC_WV2_5', 'DivAC_WV1_6',
       'DivAC_WV2_6', 'DivAC_WV1_7', 'DivAC_WV2_7', 'DivAC_WV1_8',
       'DivAC_WV2_8', 'DivAC_WV1_9', 'DivAC_WV2_9', 'DivAC_WV1_10',
       'DivAC_WV2_10', 'DivAC_WV1_11', 'DivAC_WV2_11', 'DivAC_WV1_12',
       'DivAC_WV2_12', 'DivAC_WV1_13', 'DivAC_WV2_13', 'DivAC_WV1_14',
       'DivAC_WV2_14', 'DivAC_WV1_15', 'DivAC_WV2_15', 'DivAC_WV1_16',
       'DivAC_WV2_16', 'DivAC_WV1_17', 'DivAC_WV2_17', 'DivAC_WV1_18',
       'DivAC_WV2_18', 'DivAC_WV1_19', 'DivAC_WV2_19'],
      dtype='object')

In [25]:
data = pd.wide_to_long(
    data, ["DivAC_WV1", "DivAC_WV2"], i=fetal_ac_feature_builder.get_label_names(), sep="_", j="SDD_index"
).reset_index()
# data["AC_WV1"] = np.log10(np.abs(data["AC_WV1"]))
# data["AC_WV2"] = np.log10(np.abs(data["AC_WV2"]))
data["ConcDiff"] = data["Fetal Hb Concentration 0"] - data["Fetal Hb Concentration 1"]

In [27]:
data[["DivAC_WV1", "DivAC_WV2"]].describe()

,DivAC_WV1,DivAC_WV2
count,69120.000000,69120.000000
mean,1.005104,1.008564
std,0.006762,0.008368
min,0.999994,0.999993
25%,1.000047,1.000963
50%,1.003199,1.007164
75%,1.007680,1.012823
max,1.102881,1.054219


In [28]:
# data["AC by AC"] = np.log10(data["AC_WV2"])  - np.log10(data["AC_WV1"])   # Subtracting ACs since they are log10
# data["AC by AC"] = np.square(data["AC_WV2"])   - np.square(data["AC_WV1"])   # Subtracting ACs since they are log10
data["AC by AC"] = data["DivAC_WV2"] / data["DivAC_WV1"]   # Subtracting ACs since they are log10
data["AC by AC"].describe()

count    69120.000000
mean         1.003443
std          0.005054
min          0.933196
25%          1.000372
50%          1.002707
75%          1.005398
max          1.037089
Name: AC by AC, dtype: float64

In [30]:
pyg.walk(data)

Box(children=(HTML(value='<div id="ifr-pyg-1" style="height: auto">\n    <head>\n        <meta http-equiv="Con…